# I want to work with the CSVs from [STMPD](http://www.slmpd.org/Crimereports.shtml).

These spreadsheets have points that are in wkid 102696. How do we work with data in this projection using a Spatial DataFrame?

## Let's read the CSV file

In [2]:
import pandas as pd

In [3]:
crime_csv = pd.read_csv("July2018.CSV")
crime_csv['Complaint'].tail(10)

4247    18-031057
4248    18-034004
4249    18-032809
4250    17-058354
4251    17-058354
4252    18-031069
4253    17-060265
4254    18-031494
4255    18-032953
4256    18-032059
Name: Complaint, dtype: object

In [4]:
len(crime_csv)

4257

In [5]:
columns = crime_csv.columns

In [6]:
columns

Index(['Complaint', 'CodedMonth', 'DateOccur', 'FlagCrime', 'FlagUnfounded',
       'FlagAdministrative', 'Count', 'FlagCleanup', 'Crime', 'District',
       'Description', 'ILEADSAddress', 'ILEADSStreet', 'Neighborhood',
       'LocationName', 'LocationComment', 'CADAddress', 'CADStreet', 'XCoord',
       'YCoord'],
      dtype='object')

In [7]:
crime_csv.head()

,Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,Description,ILEADSAddress,ILEADSStreet,Neighborhood,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord
0,18-030787,2018-07,01/01/2006 12:00,Y,,,1,,115400,6,STLG BY DECEIT/IDENTITY THEFT REPORT,3800,N NEWSTEAD AVE,68,,,3800,NEWSTEAD,895638.1,1033127.0
1,18-030778,2018-07,01/01/2014 12:00,Y,,,1,,115400,6,STLG BY DECEIT/IDENTITY THEFT REPORT,5052,PLOVER AVE,72,,,5052,PLOVER,890976.9,1042886.0
2,18-033335,2018-07,01/01/2015 12:00,Y,,,1,,115400,6,STLG BY DECEIT/IDENTITY THEFT REPORT,8915,HALLS FERRY RD,74,,,8915,HALLS FERRY,894751.8,1050966.0
3,18-033321,2018-07,01/01/2017 12:00,Y,,,1,,115400,6,STLG BY DECEIT/IDENTITY THEFT REPORT,4864,LEE AVE,69,,,4864,LEE,893100.3,1036196.0
4,18-033370,2018-07,01/01/2017 12:00,Y,,,1,,115400,6,STLG BY DECEIT/IDENTITY THEFT REPORT,5115,ROBIN AVE,72,,,5115,ROBIN,890348.6,1043545.0


In [8]:
crime_csv.at[0, 'Complaint']

'18-030787'

## Filter 0 points

In [9]:
filtered_crime_csv = crime_csv[crime_csv['XCoord'] != 0]

In [10]:
len(filtered_crime_csv)

4157

In [11]:
filtered_crime_csv = crime_csv[(crime_csv['XCoord'] != 0) | (crime_csv['YCoord'] != 0)]
len(filtered_crime_csv)

4157

## Filter by Date

In [12]:
new_crim_df = filtered_crime_csv[filtered_crime_csv.DateOccur >= '07/01/2018']
len(new_crim_df)

3853

## Let's Turn this into a feature class using the Spatial DataFrame

In [13]:
import arcgis
from arcgis.features import SpatialDataFrame

In [14]:
wkid = 102696
crime_coords = new_crim_df.iloc[:,18:20].values.tolist()
crime_coords

[[897798.1, 1008384.0],
 [898179.0, 1046847.0],
 [893199.2, 1021854.0],
 [905041.7, 1011294.0],
 [905041.7, 1011294.0],
 [884777.1, 1003839.0],
 [894107.6, 1042683.0],
 [898034.7, 1012024.0],
 [886938.5, 1002392.0],
 [900061.2, 1018771.0],
 [906256.9, 1022439.0],
 [906666.7, 1015976.0],
 [908574.0, 1025613.0],
 [900324.4, 1036832.0],
 [898026.0, 1002536.0],
 [881087.5, 1000532.0],
 [881095.4, 999574.9],
 [883631.8, 1028675.0],
 [894686.4, 1049981.0],
 [881921.9, 999270.2],
 [893905.0, 1007810.0],
 [895809.9, 1018289.0],
 [891808.7, 1022061.0],
 [907779.0, 1022680.0],
 [903630.3, 1014241.0],
 [908223.8, 1016086.0],
 [895016.4, 1036987.0],
 [883224.6, 1032632.0],
 [899523.6, 1009803.0],
 [887396.8, 1020150.0],
 [895152.2, 1037234.0],
 [888035.9, 986508.1],
 [903145.3, 1014215.0],
 [894902.8, 1015747.0],
 [884846.6, 1013261.0],
 [906212.9, 1020777.0],
 [903550.3, 1010232.0],
 [887701.6, 1044511.0],
 [899955.5, 1029968.0],
 [903721.0, 1014843.0],
 [889296.1, 1023305.0],
 [898194.8, 1005777

## Convert to list of geometries

In [37]:
geometries=[arcgis.geometry.Geometry({'x':r[0], 'y':r[1], 
                    'spatialReference':{'wkid':wkid}}) for r in crime_coords]

In [17]:
from arcgis.features import GeoAccessor as ga

In [40]:
sdf = ga.from_xy(new_crim_df, 'XCoord', 'YCoord', sr=wkid)

C:\Users\greg6750\AppData\Local\ESRI\conda\envs\rtree-env\lib\site-packages\arcgis\features\geo\_io\fileops.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SHAPE'] = geoms
C:\Users\greg6750\AppData\Local\ESRI\conda\envs\rtree-env\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\greg6750\AppData\Local\ESRI\conda\envs\rtree-env\lib\site-packages\arcgis\features\geo\_accessor.py:1048: SettingWithCopyWarning: 
A value is trying to

In [42]:
len(sdf)

3853

In [43]:
sdf.head()

,Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,...,ILEADSAddress,ILEADSStreet,Neighborhood,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord,SHAPE
347,18-030020,2018-07,07/01/2018 00:10,Y,,,1,,265321,3,...,0,GRAVOIS AVE / PESTALOZZI ST,30,,,,,897798.1,1008384.0,"{'x': 897798.1, 'y': 1008384.0, 'spatialRefere..."
348,18-030012,2018-07,07/01/2018 00:30,Y,,,1,,265321,6,...,0,BLASE AVE / HALL ST,74,,,7600,HALL,898179.0,1046847.0,"{'x': 898179.0, 'y': 1046847.0, 'spatialRefere..."
349,18-030018,2018-07,07/01/2018 00:30,Y,,,1,,41011,5,...,4045,LINDELL BLVD,38,WING STOP,,,,893199.2,1021854.0,"{'x': 893199.2, 'y': 1021854.0, 'spatialRefere..."
350,18-030017,2018-07,07/01/2018 00:35,Y,,,1,,91113,3,...,1711,S 9TH ST,21,EPIC PIZZA,EPIC PIZZA,1711,9TH,905041.7,1011294.0,"{'x': 905041.7, 'y': 1011294.0, 'spatialRefere..."
351,18-030017,2018-07,07/01/2018 00:35,Y,,,1,,91113,3,...,1711,S 9TH ST,21,EPIC PIZZA,EPIC PIZZA,1711,9TH,905041.7,1011294.0,"{'x': 905041.7, 'y': 1011294.0, 'spatialRefere..."


## Let's see the result on a map!!!

In [44]:
from arcgis.gis import GIS
from arcgis.geoenrichment import *

gis = GIS("http://slustl.maps.arcgis.com/home", "gregbrunner_slugis")

Enter password: ········


In [45]:
stl_map = gis.map('St. Louis, MO')
stl_map

MapView(layout=Layout(height='400px', width='100%'))

### This Fails

In [46]:
crime_map = gis.content.import_data(sdf, title='STL July Crime 04042020') 
                                    #, {'Title':'July 2018 Crime Data'})

C:\Users\greg6750\AppData\Local\ESRI\conda\envs\rtree-env\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [47]:
crime_map

<Item title:"STL July Crime 04042020" type:Feature Layer Collection owner:gregbrunner_slugis>

In [48]:
stl_map.add_layer(crime_map, {"renderer":"HeatmapRenderer",
               "opacity": 0.75})

## We can also export this data to a feature class

In [51]:
sdf.spatial.to_featureclass(
    r"C:\Users\greg6750\Documents\IPython Notebooks\intro-prog-for-gis-rs\april_20\test.shp", 
    overwrite=True)

C:\Users\greg6750\AppData\Local\ESRI\conda\envs\rtree-env\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


'C:\\Users\\greg6750\\Documents\\IPython Notebooks\\intro-prog-for-gis-rs\\april_20\\test.shp'